<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/DOWNLOAD_NETCDF_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Executando script para baixar arquivos netCDFF4 do banco de dados NOAA-GOES-16
#### Script inspired by Brian Blaylock.
#####Acess in https://gist.github.com/blaylockbk/d60f4fce15a7f0475f975fc57da9104d#file-download_goes_aws-py

In [1]:
pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.7.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cup

In [2]:
import s3fs

from google.colab import drive # Opcional
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def avaliable_products(operation: str):
    """
    Exibe os produtos GOES-16 organizados por índice, código e descrição.

    Parâmetros:
        operation (str): Operação desejada.
            - 'codes': Exibe apenas os códigos dos produtos.
            - 'description': Exibe os índices, códigos e descrições dos produtos.
    """
    products = [
        ("ABI-L1b-RadF", "Advanced Baseline Imager Level 1b Full Disk"),
        ("ABI-L1b-RadC", "Advanced Baseline Imager Level 1b CONUS"),
        ("ABI-L1b-RadM", "Advanced Baseline Imager Level 1b Mesoscale"),
        ("ABI-L2-ACHAC", "Advanced Baseline Imager Level 2 Cloud Top Height CONUS"),
        ("ABI-L2-ACHAF", "Advanced Baseline Imager Level 2 Cloud Top Height Full Disk"),
        ("ABI-L2-ACHAM", "Advanced Baseline Imager Level 2 Cloud Top Height Mesoscale"),
        ("ABI-L2-ACHTF", "Advanced Baseline Imager Level 2 Cloud Top Temperature Full Disk"),
        ("ABI-L2-ACHTM", "Advanced Baseline Imager Level 2 Cloud Top Temperature Mesoscale"),
        ("ABI-L2-ACMC", "Advanced Baseline Imager Level 2 Clear Sky Mask CONUS"),
        ("ABI-L2-ACMF", "Advanced Baseline Imager Level 2 Clear Sky Mask Full Disk"),
        ("ABI-L2-ACMM", "Advanced Baseline Imager Level 2 Clear Sky Mask Mesoscale"),
        ("ABI-L2-ACTPC", "Advanced Baseline Imager Level 2 Cloud Top Phase CONUS"),
        ("ABI-L2-ACTPF", "Advanced Baseline Imager Level 2 Cloud Top Phase Full Disk"),
        ("ABI-L2-ACTPM", "Advanced Baseline Imager Level 2 Cloud Top Phase Mesoscale"),
        ("ABI-L2-ADPC", "Advanced Baseline Imager Level 2 Aerosol Detection CONUS"),
        ("ABI-L2-ADPF", "Advanced Baseline Imager Level 2 Aerosol Detection Full Disk"),
        ("ABI-L2-ADPM", "Advanced Baseline Imager Level 2 Aerosol Detection Mesoscale"),
        ("ABI-L2-AODC", "Advanced Baseline Imager Level 2 Aerosol Optical Depth CONUS"),
        ("ABI-L2-AODF", "Advanced Baseline Imager Level 2 Aerosol Optical Depth Full Disk"),
        ("ABI-L2-CMIPC", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery CONUS"),
        ("ABI-L2-CMIPF", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery Full Disk"),
        ("ABI-L2-CMIPM", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery Mesoscale"),
        ("ABI-L2-CODC", "Advanced Baseline Imager Level 2 Cloud Optical Depth CONUS"),
        ("ABI-L2-CODF", "Advanced Baseline Imager Level 2 Cloud Optical Depth Full Disk"),
        ("ABI-L2-CPSC", "Advanced Baseline Imager Level 2 Cloud Particle Size CONUS"),
        ("ABI-L2-CPSF", "Advanced Baseline Imager Level 2 Cloud Particle Size Full Disk"),
        ("ABI-L2-CPSM", "Advanced Baseline Imager Level 2 Cloud Particle Size Mesoscale"),
        ("ABI-L2-CTPC", "Advanced Baseline Imager Level 2 Cloud Top Pressure CONUS"),
        ("ABI-L2-CTPF", "Advanced Baseline Imager Level 2 Cloud Top Pressure Full Disk"),
        ("ABI-L2-DMWC", "Advanced Baseline Imager Level 2 Derived Motion Winds CONUS"),
        ("ABI-L2-DMWF", "Advanced Baseline Imager Level 2 Derived Motion Winds Full Disk"),
        ("ABI-L2-DMWM", "Advanced Baseline Imager Level 2 Derived Motion Winds Mesoscale"),
        ("ABI-L2-DSIC", "Advanced Baseline Imager Level 2 Derived Stability Indices CONUS"),
        ("ABI-L2-DSIF", "Advanced Baseline Imager Level 2 Derived Stability Indices Full Disk"),
        ("ABI-L2-DSIM", "Advanced Baseline Imager Level 2 Derived Stability Indices Mesoscale"),
        ("ABI-L2-DSRC", "Advanced Baseline Imager Level 2 Downward Shortwave Radiation CONUS"),
        ("ABI-L2-DSRF", "Advanced Baseline Imager Level 2 Downward Shortwave Radiation Full Disk"),
        ("ABI-L2-DSRM", "Advanced Baseline Imager Level 2 Downward Shortwave Radiation Mesoscale"),
        ("ABI-L2-FDCC", "Advanced Baseline Imager Level 2 Fire (Hot Spot Characterization) CONUS"),
        ("ABI-L2-FDCF", "Advanced Baseline Imager Level 2 Fire (Hot Spot Characterization) Full Disk"),
        ("ABI-L2-LSTC", "Advanced Baseline Imager Level 2 Land Surface Temperature CONUS"),
        ("ABI-L2-LSTF", "Advanced Baseline Imager Level 2 Land Surface Temperature Full Disk"),
        ("ABI-L2-LSTM", "Advanced Baseline Imager Level 2 Land Surface Temperature Mesoscale"),
        ("ABI-L2-LVMPC", "Advanced Baseline Imager Level 2 Legacy Vertical Moisture Profile CONUS"),
        ("ABI-L2-LVMPF", "Advanced Baseline Imager Level 2 Legacy Vertical Moisture Profile Full Disk"),
        ("ABI-L2-LVMPM", "Advanced Baseline Imager Level 2 Legacy Vertical Moisture Profile Mesoscale"),
        ("ABI-L2-LVTPC", "Advanced Baseline Imager Level 2 Legacy Vertical Temperature Profile CONUS"),
        ("ABI-L2-LVTPF", "Advanced Baseline Imager Level 2 Legacy Vertical Temperature Profile Full Disk"),
        ("ABI-L2-LVTPM", "Advanced Baseline Imager Level 2 Legacy Vertical Temperature Profile Mesoscale"),
        ("ABI-L2-MCMIPC", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery CONUS"),
        ("ABI-L2-MCMIPF", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery Full Disk"),
        ("ABI-L2-MCMIPM", "Advanced Baseline Imager Level 2 Cloud and Moisture Imagery Mesoscale"),
        ("ABI-L2-RRQPEF", "Advanced Baseline Imager Level 2 Rainfall Rate (QPE) Full Disk"),
        ("ABI-L2-RSRC", "Advanced Baseline Imager Level 2 Reflected Shortwave Radiation TOA CONUS"),
        ("ABI-L2-RSRF", "Advanced Baseline Imager Level 2 Reflected Shortwave Radiation TOA Full Disk"),
        ("ABI-L2-SSTF", "Advanced Baseline Imager Level 2 Sea Surface (Skin) Temperature Full Disk"),
        ("ABI-L2-TPWC", "Advanced Baseline Imager Level 2 Total Precipitable Water CONUS"),
        ("ABI-L2-TPWF", "Advanced Baseline Imager Level 2 Total Precipitable Water Full Disk"),
        ("ABI-L2-TPWM", "Advanced Baseline Imager Level 2 Total Precipitable Water Mesoscale"),
        ("ABI-L2-VAAF", "Advanced Baseline Imager Level 2 Volcanic Ash Detection Full Disk"),
        ("GLM-L2-LCFA", "Geostationary Lightning Mapper Level 2 Lightning Detection"),
        ("SUVI-L1b-Fe093", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
        ("SUVI-L1b-Fe131", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
        ("SUVI-L1b-Fe171", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
        ("SUVI-L1b-Fe195", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
        ("SUVI-L1b-Fe284", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
        ("SUVI-L1b-He303", "Solar Ultraviolet Imager Level 1b Extreme Ultraviolet"),
    ]

    if operation == 'codes':

        return [code for code, _ in products]

    if operation == 'description':

        print(f"{'Index':<5} | {'Produto':<25} | {'Descrição'}")

        print("-" * 95)

        for i, (code, description) in enumerate(products):

            print(f"{i:<5} | {code:<25} | {description}")

def get_netcdf_data(product: str, year_start: int, year_end: int, day_start: int, day_end: int, hour: str):
    """
    Obtém arquivos NetCDF4 da AWS S3 (NOAA GOES-16) com base no intervalo de datas e hora.

    Parâmetros:
        product (str): Produto desejado (ex: 'ABI-L2-CMIPF').
        year_start (int): Ano inicial.
        year_end (int): Ano final.
        day_start (int): Dia inicial (ex: 152).
        day_end (int): Dia final (ex: 212).
        hour (str): Hora do dia (formato: 'HH', ex: '16').

    Retorna:
        Tuple:
            files (List[List[str]]): Lista de listas com caminhos dos arquivos encontrados.
            errors (List[Tuple[int, int]]): Lista de (ano, dia) onde houve erro de leitura.
    """
    fs = s3fs.S3FileSystem(anon=True)
    files = []
    errors = []

    for year in range(year_start, year_end + 1):

        for day in range(day_start, day_end + 1):

            try:

                path = f'noaa-goes16/{product}/{year}/{day:03}/{hour}/'

                files.append(fs.ls(path))

            except Exception:

                errors.append((year, day))

    return files, errors

def detect_has_bands(data: list) -> bool:
    """
    Verifica se os arquivos possuem bandas do tipo M6C##.
    """
    for daily_files in data:

        for file in daily_files:

            if "M6C" in file:

                return True

    return False

def filter_files(data: list) -> list:
    """
    Filtra os arquivos:
      - Se tiver bandas, retorna SOMENTE os arquivos das bandas M6C01, M6C02, M6C03, M6C05 e M6C06.
      - Se não tiver bandas, retorna TODOS os arquivos por dia.
    Retorna uma lista plana de caminhos de arquivos.
    """
    has_bands = detect_has_bands(data)
    filtered = []
    bandas_selecionadas = ["M6C01", "M6C02", "M6C03", "M6C05", "M6C06"]  # Bandas desejadas

    if has_bands:
        for daily_files in data:
            # Filtra apenas as bandas selecionadas
            for banda in bandas_selecionadas:
                matched = [f for f in daily_files if banda in f]
                filtered.extend(matched)  # Adiciona TODOS os arquivos desta banda
    else:
        # Se não houver bandas, adiciona todos os arquivos do dia
        for daily_files in data:
            if daily_files:
                filtered.extend(daily_files)

    return filtered

def download_files(product: str, file_list: list, local_base_dir: str):
    """
    Faz o download dos arquivos NetCDF filtrados para o diretório local.

    Parâmetros:
        product (str): Nome do produto (ex: 'ABI-L2-CMIPF').
        file_list (list): Lista de caminhos dos arquivos no S3.
        local_base_dir (str): Caminho base local para salvar os arquivos.
    """
    fs = s3fs.S3FileSystem(anon=True)

    for idx, remote_path in enumerate(file_list):

        filename = remote_path.split('/')[-1]

        local_path = f'{local_base_dir}/{product}/netCDF/{filename}'

        try:

            fs.get(remote_path, local_path)

            print(f'Arquivo {idx + 1}/{len(file_list)} baixado: {filename}')

        except Exception as e:

            print(f'Erro ao baixar {filename}: {e}')

    print('Todos os downloads foram concluídos.')

# Exemplo de uso:
product = avaliable_products('codes')[39] # 9 - ACMF, 20 - CMIPF, 39 - FDCF
files = get_netcdf_data(product, 2020, 2020, 152, 213, '13')[0]
filtered = filter_files(files)
path = '/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS'
download_files(product, filtered, path)